In [1]:
import json
import os
import subprocess
from IPython.display import clear_output

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cuda


# Generate metrics

In [2]:
dic_tarif_zero = {'v_sent': 0,
                  'v_sent_full': 0,
            'v_degraded':0,
            'cancelled':0, #cancel departure
            'function_not_found':0,
            'v1_not_sent_from_1st_station':0,
            'v_not_found_in_last_station':0,
            # 'ff_sent':0,
            # 'ff_skill_mean':0,
            'z1_sent': 0,
            'skill_lvl':0
            }

In [3]:
os.chdir('./Reward_weights')

with open("rw_v_not_found_in_last_station_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_sent': 0,
 'v_sent_full': 0,
 'v_degraded': 0,
 'cancelled': 0,
 'function_not_found': 0,
 'v1_not_sent_from_1st_station': 0,
 'v_not_found_in_last_station': -100,
 'z1_sent': 0,
 'skill_lvl': 0}

In [4]:
os.chdir('./Reward_weights')

with open("rw_z1_sent_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_sent': 0,
 'v_sent_full': 0,
 'v_degraded': 0,
 'cancelled': 0,
 'function_not_found': 0,
 'v1_not_sent_from_1st_station': 0,
 'v_not_found_in_last_station': 0,
 'z1_sent': -100,
 'skill_lvl': 0}

# Agent params

In [30]:
cmd = ["python3", "-u", "agent_pomo_run.py"]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
for line in process.stdout:
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

Nombre d'agent = 2. on démarre le multi process ...
max_train_steps 265435
lr decay: 1 decay_update: 100 PER 1
max_train_steps 265435
lr decay: 1 decay_update: 100 PER 1
Dueling_QNetwork(
(model): Sequential(
(0): Linear(in_features=3280, out_features=1024, bias=True)
(1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(2): ReLU()
(3): Linear(in_features=1024, out_features=1024, bias=True)
(4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(5): ReLU()
(6): Linear(in_features=1024, out_features=1024, bias=True)
(7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(8): ReLU()
(9): Linear(in_features=1024, out_features=1024, bias=True)
(10): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(11): ReLU()
(12): Linear(in_features=1024, out_features=1024, bias=True)
(13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(14)

KeyboardInterrupt: 